In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    # demke - fill in as we see what state gets generated in this page.
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    #''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()



In [2]:
appdir=os.getenv('HOME')
appdir=appdir + "/sync"
output = runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";cp -r ../src/sync " + appdir)

bash = BashSession(cwd=appdir)

In [3]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/bounded_buffer_wq.c", 
    lang="", 
    number=True,
    title="<b>Bounded buffer with sleep() and wakeup()</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>Bounded buffer with sleep() and wakeup()</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: /* For illustration purposes only. 
1:  * This code is not complete and cannot be compiled into an executable.
2:  */
3: #include <stdatomic.h>
4: #include <stdbool.h>
5: #include "waitqueue.h"
6: 
7: extern char produce_item();
8: extern void consume_item(char item);
9: 
10: #define N 2
11: 
12: struct bounded_buffer {
13: 	char data[N];   /* 'items' in this example are just bytes */
14: 	int in;     /* index where producer inserts items, initially 0 */
15: 	int out;    /* index where consumer removes items, initially 0 */
16: 	atomic_int count;  /* number of items currently in buffer, initially 0 */
17: 	struct wait_queue waitq; /* keep track of waiting threads */
18: };
19: 
20: struct bounded_buffer bb;
21: 
22: void *producer(void *arg) {
23:     while(true) {
24:         char item = produce_item();
25:         if (bb.count == N)
26:             sleep(&bb.waitq);
27:         bb.data[bb.in] = item;
28:         bb.in = (bb.in + 1) % N;
29:         bb.count++;
30:         if (bb.count == 1) 
31:             wakeup(&bb.waitq);
32:     }
33: }
34: 
35: void *consumer(void *arg) {
36:     while(true) {
37:         if (bb.count == 0) 
38:             sleep(&bb.waitq);
39:         char item = bb.data[bb.out];
40:         bb.out = (bb.out + 1) % N;
41:         bb.count--;
42:         if (bb.count == N-1) 
43:             wakeup(&bb.waitq);
44:         consume_item(item);
45:     }
46: }

```


</div>
</font>

In [4]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/bounded_buffer_sem.c", 
    lang="", 
    number=True,
    title="<b>Bounded buffer with semaphores</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>Bounded buffer with semaphores</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: /* For illustration purposes only. 
1:  * This code is not complete and cannot be compiled into an executable.
2:  */
3: #include <semaphore.h>
4: #include <stdbool.h>
5: 
6: extern char produce_item();
7: extern void consume_item(char item);
8: 
9: #define N 2
10: 
11: struct bounded_buffer {
12:     char data[N];   /* 'items' in this example are just bytes */
13:     int in;     /* index where producer inserts items, initially 0 */
14:     int out;    /* index where consumer removes items, initially 0 */
15:     sem_t sem_empty;
16:     sem_t sem_filled;
17: };
18: 
19: struct bounded_buffer bb;
20: 
21: void *producer(void *arg) {
22:     while(true) {
23:         char item = produce_item();
24:         sem_wait(&bb.sem_empty);
25:         bb.data[bb.in] = item;
26:         bb.in = (bb.in + 1) % N;
27:         sem_post(&bb.sem_filled);
28:     }
29: }
30: 
31: void *consumer(void *arg) {
32:     while(true) {
33:         sem_wait(&bb.sem_filled);
34:         char item = bb.data[bb.out];
35:         bb.out = (bb.out + 1) % N;
36:         sem_post(&bb.sem_empty);
37:         consume_item(item);
38:     }
39: }
40: 
41: int main()
42: {
43:     sem_init(&bb.sem_empty, 0, N);
44:     sem_init(&bb.sem_filled,0, 0);
45:     
46:     /*** Thread creation stuff here ***/
47:     
48:     return 0;
49: }

```


</div>
</font>

In [5]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/bounded_buffer_sem_multi.c", 
    lang="", 
    number=True,
    title="<b>Bounded buffer with semaphores for multiple producers and consumers.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>Bounded buffer with semaphores for multiple producers and consumers.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: /* For illustration purposes only. 
1:  * This code is not complete and cannot be compiled into an executable.
2:  */
3: #include <semaphore.h>
4: #include <stdbool.h>
5: 
6: extern char produce_item();
7: extern void consume_item(char item);
8: 
9: #define N 2
10: 
11: struct bounded_buffer {
12: 	char data[N];   /* 'items' in this example are just bytes */
13: 	int in;     /* index where producer inserts items, initially 0 */
14: 	int out;    /* index where consumer removes items, initially 0 */
15:     sem_t sem_empty;
16:     sem_t sem_filled;
17:     sem_t sem_mutex;
18: };
19: 
20: struct bounded_buffer bb;
21: 
22: void *producer(void *arg) {
23:     while(true) {
24:         char item = produce_item();
25:         sem_wait(&bb.sem_empty);
26:         sem_wait(&bb.sem_mutex);
27:         bb.data[bb.in] = item;
28:         bb.in = (bb.in + 1) % N;
29:         sem_post(&bb.sem_mutex);
30:         sem_post(&bb.sem_filled);
31:     }
32: }
33: 
34: void *consumer(void *arg) {
35:     while(true) {
36:         sem_wait(&bb.sem_filled);
37:         sem_wait(&bb.sem_mutex);
38:         char item = bb.data[bb.out];
39:         bb.out = (bb.out + 1) % N;
40:         sem_post(&bb.sem_mutex);
41:         sem_post(&bb.sem_empty);
42:         consume_item(item);
43:     }
44: }
45: 
46: int main()
47: {
48:     sem_init(&bb.sem_empty, 0, N);
49:     sem_init(&bb.sem_filled, 0, 0);
50:     sem_init(&bb.sem_mutex, 0, 1);
51:     
52:     /*** Thread creation stuff here ***/
53:     
54:     return 0;
55: }

```


</div>
</font>

In [6]:
# demke:
# 
# This cell is removed in the html, but displays the code in the Jupyter notebook.
#

display(Markdown('<font size="1.2rem">' + FileCodeBox(
    file=appdir + "/bounded_buffer_cv.c", 
    lang="", 
    number=True,
    title="<b>Bounded buffer with condition variables for multiple producers and consumers.</b>",
    h="100%", 
    w="100%"
) + '</font>'))

<font size="1.2rem"><b>Bounded buffer with condition variables for multiple producers and consumers.</b>
<div style="width:100%; height:100%; font-size:inherit; overflow: auto;" >


``` 
0: /* For illustration purposes only. 
1:  * This code is not complete and cannot be compiled into an executable.
2:  */
3: #include <pthread.h>
4: #include <stdbool.h>
5: 
6: extern char produce_item();
7: extern void consume_item(char item);
8: 
9: #define N 2
10: 
11: struct bounded_buffer {
12:     char data[N];   /* 'items' in this example are just bytes */
13:     int in;     /* index where producer inserts items, initially 0 */
14:     int out;    /* index where consumer removes items, initially 0 */
15:     int count;
16:     pthread_mutex_t buflock;
17:     pthread_cond_t not_full;
18:     pthread_cond_t not_empty;
19: };
20: 
21: struct bounded_buffer bb;
22: 
23: void *producer(void *arg) {
24:     while(true) {
25:         char item = produce_item();
26:         pthread_mutex_lock(&bb.buflock);
27:         while(bb.count == N) {
28:             pthread_cond_wait(&bb.notfull, &bb.buflock);
29:         }
30:         bb.data[bb.in] = item;
31:         bb.in = (bb.in + 1) % N;
32:         bb.count++;
33:         pthread_cond_signal(&bb.notempty);
34:         pthread_mutex_unlock(&bb.buflock);
35:     }
36: }
37: 
38: void *consumer(void *arg) {
39:     while(true) {
40:         pthread_mutex_lock(&bb.buflock);
41:         while (bb.count == 0) {
42:             pthread_cond_wait(&bb.notempty, &bb.buflock);
43:         }
44:         char item = bb.data[bb.out];
45:         bb.out = (bb.out + 1) % N;
46:         bb.count--;
47:         pthread_cond_signal(&bb.notfull);
48:         pthread_mutex_unlock(&bb.buflock);
49: 
50:         consume_item(item);
51:     }
52: }
53: 
54: int main()
55: {
56:     pthread_mutex_init(&bb.buflock, NULL);
57:     pthread_cond_init(&bb.notfull, NULL);
58:     pthread_cond_init(&bb.notempty, NULL);
59:     
60:     /*** Thread creation stuff here ***/
61:     
62:     return 0;
63: }

```


</div>
</font>